📢 [프롬프트 엔지니어링, LLM(ChatGPT) 기반의 AI 서비스 개발](https://fastcampus.co.kr/data_red_golbin) 강의에서 사용하는 예제입니다.

### 데이터 생성 프롬프트

```
첨부한 기고문에서 질문과 답변 세트를 만들어줘

--- 기고문 ---
{article}
--- 기고문 끝 ---

질문과 답변 세트를 5개 만들어줘.
각 질문과 답변은 전부 다른 내용을 담고 있어야 해.
숫자로 반복문을 돌리지 말고, 첨부한 파일의 내용에 있는 내용을 구체적으로 담아야 해.

질문 답변 필드명은 instruction과 output으로 작성해줘
질문과 답변은 한국어로 작성해줘

출력 형식은 CSV로 출력해줘. 각 항목은 쌍따옴표로 감싸줘.
```

생성한 파일

- CSV 데이터 파일: [the-age-of-AI-has-begun.csv](https://drive.google.com/file/d/13G40JFsrSJB8GgZI4AiC2s0yF3tmVQCj/view?usp=sharing)
- 기고문 텍스트 파일: [The Age of AI has begun](https://docs.google.com/document/d/1nXYbAJRYzQ9To1pXscFZ10YU_5G2A1N-ml4_kbaGIJY/edit?usp=sharing)

In [1]:
!nvidia-smi

Tue Aug 15 19:03:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 모델 로딩

In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

## 데이터셋 로딩

In [4]:
from datasets import load_dataset

data = load_dataset("csv",
                    data_files={"train":"the-age-of-AI-has-begun.csv"})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
[row for row in data['train']]

[{'instruction': 'AI가 혁신적인 기술이라고 생각하시나요?',
  'output': '네, AI는 혁신적인 기술입니다. 인공지능은 모바일 폰과 인터넷과 마찬가지로 혁명적인 영향을 미칠 것으로 기대됩니다.'},
 {'instruction': '1980년대에 첫 번째 혁신적인 기술을 보셨다고 하셨는데, 어떤 기술이었나요?',
  'output': '1980년대에는 그래픽 사용자 인터페이스를 처음 보았습니다. 이는 현대의 운영 체제를 포함한 모든 운영 체제의 전신이었습니다.'},
 {'instruction': 'AI가 어떻게 교육 분야에서 불평등을 줄일 수 있는지 생각하셨나요?',
  'output': 'AI는 교육 분야에서 불평등을 줄일 수 있는 다양한 방법이 있습니다. 예를 들어, AI 기반 소프트웨어는 학생의 학습 스타일과 관심사를 파악하여 맞춤형 콘텐츠를 제공할 수 있습니다.'},
 {'instruction': 'AI가 어떻게 의료 분야에서 도움을 줄 수 있을까요?',
  'output': 'AI는 의료진의 시간을 절약하고 보험 청구, 서류 작성 등과 같은 일부 업무를 처리하는 데 도움을 줄 수 있습니다. 또한 AI는 환자들이 기본적인 응급 처치 방법에 대한 조언을 받을 수 있도록 도와줄 수 있습니다.'},
 {'instruction': 'AI가 어떻게 농업 분야에서 도움을 줄 수 있을까요?',
  'output': 'AI는 농업 분야에서 다양한 방식으로 도움을 줄 수 있습니다. 예를 들어, AI는 토양과 날씨에 기반하여 농부에게 가장 적합한 종자를 조언하거나 가축에 대한 약물 및 백신을 개발하는 데 도움을 줄 수 있습니다.'}]

In [6]:
data = data.map(
    lambda x: {
        'text': f"""User: {x['instruction']}

Assistant: {x['output']}<|endoftext|>""" }
)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [7]:
[row['text'] for row in data['train']]

['User: AI가 혁신적인 기술이라고 생각하시나요?\n\nAssistant: 네, AI는 혁신적인 기술입니다. 인공지능은 모바일 폰과 인터넷과 마찬가지로 혁명적인 영향을 미칠 것으로 기대됩니다.<|endoftext|>',
 'User: 1980년대에 첫 번째 혁신적인 기술을 보셨다고 하셨는데, 어떤 기술이었나요?\n\nAssistant: 1980년대에는 그래픽 사용자 인터페이스를 처음 보았습니다. 이는 현대의 운영 체제를 포함한 모든 운영 체제의 전신이었습니다.<|endoftext|>',
 'User: AI가 어떻게 교육 분야에서 불평등을 줄일 수 있는지 생각하셨나요?\n\nAssistant: AI는 교육 분야에서 불평등을 줄일 수 있는 다양한 방법이 있습니다. 예를 들어, AI 기반 소프트웨어는 학생의 학습 스타일과 관심사를 파악하여 맞춤형 콘텐츠를 제공할 수 있습니다.<|endoftext|>',
 'User: AI가 어떻게 의료 분야에서 도움을 줄 수 있을까요?\n\nAssistant: AI는 의료진의 시간을 절약하고 보험 청구, 서류 작성 등과 같은 일부 업무를 처리하는 데 도움을 줄 수 있습니다. 또한 AI는 환자들이 기본적인 응급 처치 방법에 대한 조언을 받을 수 있도록 도와줄 수 있습니다.<|endoftext|>',
 'User: AI가 어떻게 농업 분야에서 도움을 줄 수 있을까요?\n\nAssistant: AI는 농업 분야에서 다양한 방식으로 도움을 줄 수 있습니다. 예를 들어, AI는 토양과 날씨에 기반하여 농부에게 가장 적합한 종자를 조언하거나 가축에 대한 약물 및 백신을 개발하는 데 도움을 줄 수 있습니다.<|endoftext|>']

## 텍스트 데이터 tokenize

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [63]:
data["train"][0]["text"]

'User: AI가 혁신적인 기술이라고 생각하시나요?\n\nAssistant: 네, AI는 혁신적인 기술입니다. 인공지능은 모바일 폰과 인터넷과 마찬가지로 혁명적인 영향을 미칠 것으로 기대됩니다.<|endoftext|>'

In [24]:
tokenized_text = tokenizer(data["train"][0]["text"])
tokenized_text

{'input_ids': [56, 426, 85, 29, 4312, 293, 2253, 407, 333, 1335, 1420, 893, 284, 344, 4389, 34, 202, 202, 36, 12732, 15097, 15741, 29, 793, 15, 4312, 272, 2253, 407, 333, 1335, 1441, 17, 5626, 296, 3049, 13196, 359, 2468, 359, 3199, 286, 6812, 407, 333, 1721, 276, 6120, 388, 339, 1321, 10023, 17, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [64]:
token_text = [tokenizer.decode(id) for id in tokenized_text["input_ids"]]
", ".join(token_text)

'U, se, r, :,  AI, 가,  혁신, 적, 인,  기술, 이라고,  생각, 하, 시, 나요, ?, \n, \n, A, ss, ist, ant, :,  네, ,,  AI, 는,  혁신, 적, 인,  기술, 입니다, .,  인공지능, 은,  모바일,  폰, 과,  인터넷, 과,  마찬가지, 로,  혁명, 적, 인,  영향, 을,  미칠,  것, 으로,  기대, 됩니다, ., <|endoftext|>'

## PEFT 학습 준비

In [30]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [32]:
# 학습되는 파라미터 수 출력
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"""trainable params: {trainable_params}
all params: {all_param}
trainable%: {100 * trainable_params / all_param}""")

trainable params: 6553600
all params: 6608701440
trainable%: 0.09916622894073424


## 학습하기

In [53]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        max_steps=100, # 100 step 학습시, V100 기준 약 2분 소요
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        learning_rate=1e-4,
        fp16=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        logging_steps=25,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

Step,Training Loss
25,1.262600
50,0.238200
75,0.157300
100,0.135000


TrainOutput(global_step=100, training_loss=0.44827259302139283, metrics={'train_runtime': 106.3388, 'train_samples_per_second': 1.881, 'train_steps_per_second': 0.94, 'total_flos': 503969429483520.0, 'train_loss': 0.44827259302139283, 'epoch': 33.33})

In [54]:
model.eval()
model.config.use_cache = True

In [55]:
from transformers.generation import GenerationConfig

gen_cfg = GenerationConfig.from_model_config(model.config)
gen_cfg.pad_token_id = tokenizer.eos_token_id

def gen(x):
    gen_cfg.temperature = 0.3
    gen_cfg.max_new_tokens = 128

    gened = model.generate(
        **tokenizer(
            f"User: {x}\n\nAssistant:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        do_sample=True,
        generation_config=gen_cfg
    )

    return tokenizer.decode(gened[0])

In [62]:
gen('1980년대에 첫 번째 혁신적인 기술을 보셨다고 하셨는데, 어떤 기술이었나요?')

'User: 1980년대에 첫 번째 혁신적인 기술을 보셨다고 하셨는데, 어떤 기술이었나요?\n\nAssistant: 1980년대에는 그래픽 사용자 인터페이스를 처음 보았습니다. 이는 현대의 운영 체제를 포함한 모든 운영 체제의 전신이었습니다. 운영 체제는 사용자가 프로그램과 연락처와 같은 기본적인 유틸리티를 포함하여 모든 운영 체제에서 기본적으로 사용할 수 있는 기본 유틸리티를 포함한 모든 운영 체제에서 기본적으로 사용할 수 있는 기본 유틸리티를 포함한 모든 운영 체제를 포함한 모든 운영 체제를 포함한 모든 운영 체제를 포함한 모든 운영 체제를 포함한 모든 운영 체제를 포함한 모든 운영 체제를 포함한 모든 운영 체제를 포함한'

## 모델 저장 & 업로드

In [41]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [42]:
model.push_to_hub('golbin/koalpaca-polyglot-12.8b-bill')

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/golbin/koalpaca-polyglot-12.8b-bill/commit/b5591d3d4fda960320dfa2fa26da2676120d2207', commit_message='Upload model', commit_description='', oid='b5591d3d4fda960320dfa2fa26da2676120d2207', pr_url=None, pr_revision=None, pr_num=None)

Reference: [KoAlpaca QLoRA Training Original Notebook by Beomi](https://colab.research.google.com/gist/Beomi/a3032e4eaa33b86fdf8de1f47f15a647/2023_05_26_bnb_4bit_koalpaca_v1_1a_on_polyglot_ko_12_8b.ipynb)